# Assignment 3 on Natural Language Processing

## Date : 23rd October, 2021

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Desai

The central idea of this assignment is to experiment  Name Entity Recogniztion using pretrained language model BERT. Please split the NER data with train and validation, and evaluate the model on the validation.

Please submit with outputs. Submissions without predicted outputs will be penalized. **Please install necessary packages according to the task.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 


import os
#print(os.listdir("../input"))
!pip install pytorch-pretrained-bert
!pip install seqeval

# Any results you write to the current directory are saved as output.

     |████████████████████████████████| 123 kB 5.4 MB/s 
     |████████████████████████████████| 131 kB 24.7 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 8.1 MB 37.7 MB/s 
     |████████████████████████████████| 138 kB 46.9 MB/s 
     |████████████████████████████████| 127 kB 47.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=0dffcbcb52d13755fba4579ba60aa05bfc031be2d908922646ed7e5c5dd4682c
  Stored in directory

Read the data

In [3]:
dframe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Assignment-3/A3_dataset.csv', encoding = "ISO-8859-1", error_bad_lines=False)


b'Skipping line 281837: expected 25 fields, saw 34\n'


In [4]:
dframe.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [5]:
dataset = dframe.drop_duplicates()
dataset = dataset.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word','shape'],axis=1)


#dataset = dframe.dropdframe.drop([])

In [6]:
dataset.head()

,pos,sentence_idx,word,tag
0,NNS,1.0,Thousands,O
1,IN,1.0,of,O
2,NNS,1.0,demonstrators,O
3,VBP,1.0,have,O
4,VBN,1.0,marched,O


Preprocess the dataset

In [7]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w,p, t) for w,p, t in zip(s["word"].values.tolist(),
                                                       s['pos'].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
getter = SentenceGetter(dataset)

In [9]:

sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [10]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [11]:
tags_vals = list(set(dataset["tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [12]:
print(tag2idx)

{'I-geo': 0, nan: 1, 'I-per': 2, 'O': 3, 'B-nat': 4, 'B-tim': 5, 'I-art': 6, 'I-tim': 7, 'I-nat': 8, 'B-eve': 9, 'B-gpe': 10, 'B-org': 11, 'I-gpe': 12, 'B-art': 13, 'I-eve': 14, 'B-per': 15, 'I-org': 16, 'B-geo': 17}


Import model pacakages

In [13]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
import random

Set up seed value

In [14]:
# setup random seed
seed =42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True


In [15]:
MAX_LEN = 40
bs =64

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [17]:
torch.cuda.get_device_name(0) 

'Tesla K80'

Features generation  for pre-trained languge BERT

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 2117152.22B/s]


In [19]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])


['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']


In [20]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [21]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [22]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [23]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [24]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [25]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [26]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

100%|██████████| 407873900/407873900 [00:11<00:00, 35376321.78B/s]


In [27]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [28]:

param_optimizer = list(model.classifier.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [29]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [30]:
epochs = 5
max_grad_norm = 1.0

print("--------------------------------------------------------Training the model...--------------------------------------------------------")

for _ in trange(epochs, desc="Epoch"):

    # TRAIN loop
    model.train()
    total_loss = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        loss = model(input_ids = b_input_ids.long(), attention_mask = b_input_mask.long(), labels = b_labels.long())
        
        # backward pass
        loss.backward()
        
        # track train loss
        total_loss += loss.item()
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = max_grad_norm)
        
        # update parameters
        optimizer.step()
    
    # print train loss per epoch
    loss_epoch = total_loss / len(train_dataloader)
    print()
    print('Epoch: {} \tTraining Loss: {:.5f}'.format(_ + 1, loss_epoch))

#save your model after the final epoch
torch.save(model, "19EE10050_NLP_Assignment_3_model.pt")
print("Model saved")
print()


print("--------------------------------------------------------Evaluating the model...--------------------------------------------------------")

# evaluate the saved model on the validation set and predict the value also

predictions = []
true_labels = []
val_accuracy_epochs = []

for batch in valid_dataloader:
    
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    predictions.append(logits)
    true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
val_accuracy = flat_accuracy(predictions, true_labels)
val_accuracy_epochs.append(val_accuracy)

print('\nEpoch: {} \tValidation Accuracy: {:.6f}'.format(_ + 1, val_accuracy))
print()

--------------------------------------------------------Training the model...--------------------------------------------------------


Epoch:  20%|██        | 1/5 [07:40<30:43, 460.79s/it]


Epoch: 1 	Training Loss: 1.80424


Epoch:  40%|████      | 2/5 [15:21<23:01, 460.65s/it]


Epoch: 2 	Training Loss: 0.83482


...

Epoch:  80%|████████  | 4/5 [30:48<07:42, 462.54s/it]


Epoch: 4 	Training Loss: 0.60112


Epoch: 100%|██████████| 5/5 [38:32<00:00, 462.41s/it]


Epoch: 5 	Training Loss: 0.57152


Model saved

--------------------------------------------------------Evaluating the model...--------------------------------------------------------

Epoch: 5 	Validation Accuracy: 0.914646



**Predictions**

In [31]:


idx2tag = {i: t for i, t in enumerate(tags_vals)}
pred_labels = np.argmax(predictions, axis=2)
pred_labels = pred_labels.flatten()
pred_labels = pred_labels.tolist()
true_labels = torch.tensor(true_labels).flatten()
true_labels = true_labels.tolist()

for preds, true_label in zip(pred_labels, true_labels):
    print("{}\t{}".format(idx2tag[preds], idx2tag[true_label]))

Streaming output truncated to the last 5000 lines.
O	O
O	B-gpe
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-geo
O	O
O	B-per
O	I-per
O	O
I-per	O
O	O
O	O
I-per	O
O	O
O	O
O	B-geo
O	I-geo
O	O
O	B-geo
O	I-geo
O	O
O	O
O	O
O	O
O	O
O	O
O	B-geo
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-per
O	I-per
O	O
O	O
O	B-org
O	O
O	B-tim
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
B-per	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-org
O	O
O	O
B-geo	B-geo
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-tim
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-org
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-gpe
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	O
O	B-g

In [34]:
#Save the  model and result

pred_labels_tags = [idx2tag[i] for i in pred_labels]
true_labels_tags = [idx2tag[j] for j in true_labels]

#Saving the predictions
df = pd.DataFrame(list(zip(true_labels_tags, pred_labels_tags)), columns=['true_labels', 'pred_labels'])
df.to_csv('19EE10050_pred_labels.csv', index=False)